<a href="https://colab.research.google.com/github/rafaqatali69/QUARTER-3/blob/main/assignment_221225_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet google-genai

In [ ]:
from google.colab import userdata
GEMINI_API_KEY:str=userdata.get('GOOGLE_API_KEY')


In [ ]:
if('GEMINI_API_KEY'):
    print("KEY FOUND")
else:
  print("KEY NOT FOUND")

In [ ]:
from google import genai
from google.genai import client
client:client=genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
model:str="gemini-2.0-flash-exp"

In [ ]:
print([attr for attr in dir(client) if not attr.startswith('_')])
print([attr for attr in dir(client.models) if not attr.startswith('_')])

In [ ]:
from google.genai.types import GenerateContentResponse
from IPython.display import display, Markdown
Response: GenerateContentResponse = client.models.generate_content(
    model=model,
    contents='how does AI work'
)
generated_text = Response.text
display(Markdown(Response.text))

In [ ]:
# COUNT TOKEN
from google.genai.types import CountTokensResponse
Result: CountTokensResponse = client.models.count_tokens(
    model=model,
    contents='whats the hightest mountain in the worldhow does AI work'
)
print(Result)

In [ ]:
# IMAGE PROMPT AND RESPONSE
!curl -o image.jpg "https://i.dawn.com/primary/2014/07/53d4ba719c131.jpg"

In [ ]:
import PIL.Image as img
img_k2 = img.open('/content/image.jpg')
display(img_k2)

In [ ]:
prompt = ("""the brief notes of this mountain. Features: History, Name the persons who submit it in winter """)

In [ ]:
Response: GenerateContentResponse = client.models.generate_content(
    model=model,
    contents=[img_k2,prompt]
)
display(Markdown(Response.text))

In [ ]:
# CHAT MODEL
response: GenerateContentResponse = client.models.generate_content(
    model=model,
    contents='Pleaes Note, My name is Rafaqat ALI'
)
display(Markdown(response.text))

In [ ]:
response: GenerateContentResponse = client.models.generate_content(
    model=model,
    contents='What is my name'
)
display(Markdown(response.text))

In [ ]:
from google.genai.chats import Chat
chat: Chat = client.chats.create(
    model=model
)
response: GenerateContentResponse = chat.send_message("My name is Rafaqat Ali")
Markdown(response.text)

In [ ]:
response: GenerateContentResponse = chat.send_message("What is my name?")
Markdown(response.text)

In [ ]:
#HOW LLM UNDERSTAND VIDEO


In [ ]:
# Load sample images
#!wget https://storage.googleapis.com/generativeai-downloads/videos/Pottery.mp4 -O Pottery.mp4 -q
#!wget https://youtu.be/WCTa9HGcFg0?si=J9gZB9HC29peFjBY -O Pottery -q
!wget /content/wallclimbing.mp4 -q


In [ ]:
# Upload the videos using the File API.
# find more details about how to use in the Get Started notebook.
# Take couple of minutes because the videos need to be processed and tokenized.

import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting video is Processing.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

#pottery_video = upload_video('Pottery.mp4')
pottery_video = upload_video('wallclimbing.mp4')

In [ ]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            please generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = pottery_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)

In [ ]:
#AUDIO TESTING

In [ ]:
from google import genai
client = genai.Client(
    api_key=GEMINI_API_KEY,
    http_options= {'api_version': 'v1alpha'}
  )
MODEL = "gemini-2.0-flash-exp"

In [ ]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

from IPython.display import display, Audio

from google import genai
from google.genai import types

async def async_enumerate(it):
  n = 0
  async for item in it:
    yield n, item
    n +=1

In [ ]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [ ]:
config={
    "generation_config": {"response_modalities": ["AUDIO"]}}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini, What you know about k-2 mountain?"
    print("> ", message, "\n")
    await session.send(message, end_of_turn=True)

    turn = session.receive()
    async for n,response in async_enumerate(turn):
      if response.data is not None:
        wav.writeframes(response.data)

        if n==0:
          print(response.server_content.model_turn.parts[0].inline_data.mime_type)
        print('.', end='')


display(Audio(file_name, autoplay=True))